In [1]:
from typing import Any
from gmpy2 import *
import random
mp_version: Any
mpz: Any

In [2]:
print('GMP version:', mp_version())

GMP version: GMP 6.3.0


In [3]:
p = mpz(2294317681846666929032827582915871147244238987997154260752547927761680407421350229668773012083293163939930095650000966848974846426935880157396423658936168303309628291888369175257564190205921538136908065555094603890577067744062235964934772618271361628841709528345147510719656157699030950075087622569709218424737738471399638675144865007958387674395112624472783726720735207890567612000139879052278793348059850324662301473079411528931593319130299609354852366107441239)
q = mpz(1802494200593124212877152220232057969820149101914917780567591041508929337113672371831686701300162645584692094587236891084394805966156657242794183368629797943879758663215081196373567865079353053014764237157020467354213814092738224751701006694341238296353856053396495458533514247995304409039019527215364796433416900272253602642367831373391296602437921809825075367037815161801942356870512197084830383932961071437649554131171040771475751447685105744841506905509803343)

In [4]:
N = p * q

In [ ]:
def benchmark_exp(n, w):
    print(f"w = {w}")
    n_w_1 = n ** (w + 1)
    base = mpz(random.randint(1,n_w_1))
    exp = mpz(random.randint(1,n ** w))
    %timeit powmod(base, exp, n_w_1)

In [7]:
benchmark_exp(N, 1)

w = 1
14.6 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
benchmark_exp(N, 3)

w = 3
129 ms ± 759 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
def benchmark_exp_fine_grained(n, l_base, l_exp):
    print(f"x^[N^{l_exp}] mod N^{l_base}")
    # assert l_base >= l_exp
    modulus = n ** l_base
    base = mpz(random.randint(1,modulus)) # type: ignore
    exp = mpz(random.randint(1,n ** l_exp)) # type: ignore
    %timeit powmod(base, exp, modulus)

In [8]:
for l_base in [2, 4]:
    for l_exp in range(1, l_base + 1):
        benchmark_exp_fine_grained(N, l_base, l_exp)

x^[N^1] mod N^2
15.6 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
x^[N^2] mod N^2
29.2 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
x^[N^1] mod N^4
44.1 ms ± 61.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
x^[N^2] mod N^4
86.8 ms ± 307 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
x^[N^3] mod N^4
129 ms ± 541 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
x^[N^4] mod N^4
171 ms ± 173 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
for l_base in [2, 4]:
    for l_exp in [3]:
        benchmark_exp_fine_grained(N, l_base, l_exp)

x^[N^3] mod N^2
42.8 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
x^[N^3] mod N^4
128 ms ± 254 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
def benchmark_exp_more_fine_grained(n, l_base, l_mod, l_exp):
    print(f"[N^{l_base}]^[N^{l_exp}] mod N^{l_mod}")
    assert l_mod >= l_exp
    modulus = n ** l_mod
    base = mpz(random.randint(1,n ** l_base)) # type: ignore
    exp = mpz(random.randint(1,n ** l_exp)) # type: ignore
    %timeit powmod(base, exp, modulus)

In [8]:
l_mod = 2
l_exp = 2
for l_base in [1, 2]:
    benchmark_exp_more_fine_grained(N, l_base, l_mod, l_exp)

[N^1]^[N^2] mod N^2
29.2 ms ± 453 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[N^2]^[N^2] mod N^2
29.2 ms ± 266 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit (N ** 4)

1.87 µs ± 162 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Each RMS multiplication in our construction requires two exponentiations:
one exponentiation in Z∗
N 4 and one in Z∗
N 2 , which require roughly 60 milliseconds and 15 milliseconds
on high-end hardware, respectively, when using a 3072-bit modulus N . Therefore, we can expect each
multiplication to take between 75 and 100 milliseconds

System uses old GMP. Might want to try the latest version to see if there are improvements (without breaking the system...):
Package: libgmp-dev
Version: 2:6.2.1+dfsg-3ubuntu1